# 🔍 Web scraping

In [113]:
# import
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

# IMDB

In [85]:
#make soup with IMDB

url = "https://www.imdb.com/search/title/?groups=top_250&sort=user_rating&count=250"

headers = {"Accept-Language": "en-US,en;q=0.5"}

response = requests.get(url,headers=headers)
#response.status_code

page_contents = response.text 

soup = BeautifulSoup(page_contents, 'html.parser')

movie = []
runtime = []
year = []
genre = []
metascore = []


for film in range (0, 250):
    movie.append(soup.find_all('div', {'class':'lister-item-content'})[film].a.text)
    runtime.append(soup.find_all('span', {'class':'runtime'})[film].text[:-3])
    year.append(soup.find_all("span", {"class":"lister-item-year"})[film].text[1:5])
    genre.append(soup.find_all('span', {'class':'genre'})[film].text.strip()) 
    metascore.append(soup.find_all('div',{'class':'ratings-imdb-rating'})[film].text.strip())

    list_of_all = [movie,runtime,year,genre,metascore]

In [86]:
# create dataframe
df = pd.DataFrame(list_of_all).transpose()

# columns's names
df.columns=["movie","runtime", "year", "genre", "metascore"]

In [87]:
df

,movie,runtime,year,genre,metascore
0,The Shawshank Redemption,142,1994,Drama,9.3
1,The Godfather,175,1972,"Crime, Drama",9.2
2,The Dark Knight,152,2008,"Action, Crime, Drama",9.0
3,The Godfather: Part II,202,1974,"Crime, Drama",9.0
4,12 Angry Men,96,1957,"Crime, Drama",9.0
...,...,...,...,...,...
245,Mr. Smith Goes to Washington,129,1939,"Comedy, Drama",8.1
246,Gone with the Wind,238,1939,"Drama, History, Romance",8.1
247,It Happened One Night,105,1934,"Comedy, Romance",8.1
248,The Passion of Joan of Arc,114,1928,"Biography, Drama, History",8.1


In [88]:
# export dataframe to CSV
df.to_csv(r'data_imdb.csv', index = False)

# ROTTEN TOMATOES

In [ ]:
#make soup with ROTTEN TOMATOES

url_base = 'https://www.rottentomatoes.com/m/'

headers = {"Accept-Language": "en-US,en;q=0.5"}

score_data = []

for name in df['name']:
    change_name = name.replace(" ", "_").replace("The ","")
    url = f'{url_base}{change_name}'
    data = requests.get(url, headers=headers)
    
    if data.status_code == 404:
        score_list = ['no score', 'no score']
        score_data.append(score_list)
        
    else:
        goodsoup = BeautifulSoup(data.text,"html.parser")
        
        audience_score = goodsoup.find("score-board")["audiencescore"]
        tomato_score = goodsoup.find("score-board")["tomatometerscore"]
        
        score_list = [audience_score, tomato_score]
        score_data.append(score_list)

In [ ]:
# rename index
index = ["audience_score","tomato_score"]

# create dataframe
df_score = pd.DataFrame(score_data,columns=index)

# view dataframe
df_score

In [ ]:
# concatenate 2 dataframes
good_df = pd.concat([df, df_score], axis=1, join='inner')

In [ ]:
# export dataframe to CSV
good_df.to_csv(r'data/data.csv', index = False)